In [2]:
!uv pip install numba

Using Python 3.12.9 environment at: /Users/swap357/Documents/dev/learning-numba/.venv
Resolved 3 packages in 99ms                                          
Installed 3 packages in 49ms                                
 + llvmlite==0.44.0
 + numba==0.61.0
 + numpy==2.1.3


In [6]:
import numpy as np
from numba import njit

In [7]:
arr = np.array([1, 2, 5, 10, 15])
clipped = np.clip(arr, 3, 12)
print(clipped)

[ 3  3  5 10 12]


In [8]:
@njit
def clip_njit():
    return np.clip(np.array([1, 2]), a_min=np.nan, a_max=np.nan)


In [9]:
print(np.clip(np.array([0, 1, 2, 3]), a_min=np.nan, a_max=2))

[nan nan nan nan]


In [10]:
print(clip_njit())

[1 2]


numba’s np.clip is not handling NaN bounds the same way as np does

In [11]:
# _np_clip_impl(a, a_min, a_max, out) 
# https://github.com/numba/numba/blob/c21aa9273ef4298392695b1f4613d29456b53e5c/numba/np/arrayobj.py#L2342
def numba_np_clip(a, a_min, a_max):
    ret = np.empty_like(a)
    a_b, a_min_b, a_max_b = np.broadcast_arrays(a, a_min, a_max)
    for index in np.ndindex(a_b.shape):
        val_a = a_b[index]
        val_a_min = a_min_b[index]
        val_a_max = a_max_b[index]
        ret[index] = min(max(val_a, val_a_min), val_a_max)

    return ret

In [12]:
print("arr:", arr)
print()

print("(arr, a_min=1, a_max=2) :")
print("numba:", numba_np_clip(arr, a_min=1, a_max=2))
print("numpy:", np.clip(arr, a_min=1, a_max=2))
print()

print("(arr, a_min=np.nan, a_max=2) :")
print("numba:", numba_np_clip(arr, a_min=np.nan, a_max=2))
print("numpy:", np.clip(arr, a_min=np.nan, a_max=2))
print()

print("(arr, a_min=1, a_max=np.nan) :")
print("numba:", numba_np_clip(arr, a_min=1, a_max=np.nan))
print("numpy:", np.clip(arr, a_min=1, a_max=np.nan))
print()

print("(arr, a_min=np.nan, a_max=np.nan) :")
print("numba:", numba_np_clip(arr, a_min=np.nan, a_max=np.nan))
print("numpy:", np.clip(arr, a_min=np.nan, a_max=np.nan))

arr: [ 1  2  5 10 15]

(arr, a_min=1, a_max=2) :
numba: [1 2 2 2 2]
numpy: [1 2 2 2 2]

(arr, a_min=np.nan, a_max=2) :
numba: [1 2 2 2 2]
numpy: [nan nan nan nan nan]

(arr, a_min=1, a_max=np.nan) :
numba: [ 1  2  5 10 15]
numpy: [nan nan nan nan nan]

(arr, a_min=np.nan, a_max=np.nan) :
numba: [ 1  2  5 10 15]
numpy: [nan nan nan nan nan]


In [19]:
def revised_np_clip(a, a_min, a_max):
    dtype = np.result_type(a, a_min, a_max)
    print('required dtype for ret based on (a, a_min and a_max): ', dtype)
    br_shape = np.broadcast(a, a_min, a_max).shape
    ret = np.empty(br_shape, dtype=dtype)
    a_b, a_min_b, a_max_b = np.broadcast_arrays(a, a_min, a_max)
    for index in np.ndindex(a_b.shape):
        val_a = a_b[index]
        val_a_min = a_min_b[index]
        val_a_max = a_max_b[index]
        # Propagate NaN if either bound is NaN.
        if np.isnan(val_a_min) or np.isnan(val_a_max):
            ret[index] = np.nan
        else:
            ret[index] = min(max(val_a, val_a_min), val_a_max)

    return ret

In [20]:
print("arr:", arr)
print()

print("(arr, a_min=1, a_max=2) :")
print("revised:", revised_np_clip(arr, a_min=1, a_max=2))
print("numpy:", np.clip(arr, a_min=1, a_max=2))
print()

print("(arr, a_min=np.nan, a_max=2) :")
print("revised:", revised_np_clip(arr, a_min=np.nan, a_max=2))
print("numpy:", np.clip(arr, a_min=np.nan, a_max=2))
print()

print("(arr, a_min=1, a_max=np.nan) :")
print("revised:", revised_np_clip(arr, a_min=1, a_max=np.nan))
print("numpy:", np.clip(arr, a_min=1, a_max=np.nan))
print()

print("(arr, a_min=np.nan, a_max=np.nan) :")
print("revised:", revised_np_clip(arr, a_min=np.nan, a_max=np.nan))
print("numpy:", np.clip(arr, a_min=np.nan, a_max=np.nan))

arr: [ 1  2  5 10 15]

(arr, a_min=1, a_max=2) :
required dtype for ret based on (a, a_min and a_max):  int64
revised: [1 2 2 2 2]
numpy: [1 2 2 2 2]

(arr, a_min=np.nan, a_max=2) :
required dtype for ret based on (a, a_min and a_max):  float64
revised: [nan nan nan nan nan]
numpy: [nan nan nan nan nan]

(arr, a_min=1, a_max=np.nan) :
required dtype for ret based on (a, a_min and a_max):  float64
revised: [nan nan nan nan nan]
numpy: [nan nan nan nan nan]

(arr, a_min=np.nan, a_max=np.nan) :
required dtype for ret based on (a, a_min and a_max):  float64
revised: [nan nan nan nan nan]
numpy: [nan nan nan nan nan]


In [21]:
revised_np_clip(arr, a_min=np.inf, a_max=np.inf)

required dtype for ret based on (a, a_min and a_max):  float64


array([inf, inf, inf, inf, inf])

In [22]:
import numpy as np
from numba import njit
@njit
def clip_njit():
    return np.clip(np.array([[1], [2]]), a_min=0, a_max=np.array([3, 4, 5]))
print(np.clip(np.array([[1], [2]]), a_min=0, a_max=np.array([3, 4, 5])))
print(clip_njit())

[[1 1 1]
 [2 2 2]]
[[1]
 [2]]


In [23]:
revised_np_clip(np.array([[1], [2]]), a_min=0, a_max=np.array([3, 4, 5]))

required dtype for ret based on (a, a_min and a_max):  int64


array([[1, 1, 1],
       [2, 2, 2]])